In [1]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
import matplotlib.pyplot as plt


from collections import defaultdict

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import networkx as nx
import numpy as np
import dwave.inspector
import dimod
from dwave.system import LeapHybridSampler
import matplotlib
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


## Quantum Part
from collections import defaultdict

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import networkx as nx
import numpy as np
import dwave.inspector
import dimod
from dwave.system import LeapHybridSampler
import matplotlib
from matplotlib import pyplot as plt

#function set
%run Functions_Set.ipynb

#Page Size
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def make_dictionary():
    Var_dictionary = {}
    # set-up a dictionary
    Total = 0
    # x input
    for e in range(2):
        Var_dictionary["x_%d"%(e)] = int(e)
    Total = len(Var_dictionary)   
    return Var_dictionary, Total

In [3]:
def show_graph(matrix):
    matfig = plt.figure(figsize=(9,9)) 
    plt.matshow(matrix,fignum=matfig.number)
    plt.colorbar()
    plt.show() 

In [4]:
def dot(*mats):
    A = mats[0]
    for B in mats[1:]:
        A = np.dot(A,B)    
    return A
        

In [5]:
def add_front(a,b):
    if len(a) < len(b):
        c = b.copy()
        c[:len(a)] += a
    else:
        c = a.copy()
        
        c[:len(b)] += b  
    return c

In [6]:
def add_end(a,b):
    if len(a) < len(b):
        c = b.copy()
        c[-len(a):] += a
    else:
        c = a.copy()
        
        c[-len(b):] += b  
    return c

In [7]:
a = np.array([1,2,3,5])
b = np.array([2,1])
print(add_front(a,b))
print(add_end(a,b))

[3 3 3 5]
[1 2 5 6]


## Objective
$$
\max 8y_{1} + 9y_{1} + 5y_{3} + 6y_{4} -15x_{1} - 10x_{2} 
$$

$$
s.t. \quad y_{1} + y_{3} \leq 1
$$

$$
\qquad \  y_{1} + y_{4} \leq 1
$$

$$
\qquad \  y_{2} + y_{3} \leq 1
$$

$$
\qquad \  y_{2} + y_{4} \leq 1
$$

$$
\qquad \  x_{1} + x_{2} \geq 1
$$

$$
\qquad \  y_{1} - x_{1} \leq 0
$$

$$
\qquad \  y_{2} - x_{1} \leq 0
$$

$$
\qquad \  y_{3} - x_{2} \leq 0
$$

$$
\qquad \  y_{4} - x_{2} \leq 0
$$

$$
\qquad \ \forall \ j \quad y_{j} \geq 0
$$

## General Structure
$$
\max h^{\intercal} \cdot y + c^{\intercal} \cdot x
$$

$$
s.t. \quad A\mathbf{x} + G \mathbf{y} \leq b
$$

$$
x \in Binary
$$

$$
y \geq 0
$$

## The Master problem

$$
\max_{x,t} \, c^{\intercal} x + t
$$

$$
s.t. \quad (b-Ax)^{\intercal}u^{k} \geq t \quad \forall k \in K
$$

$$
\qquad \ (b-Ax)^{\intercal}r^{j} \geq 0 \quad \forall j \in J
$$

$$
t \in \mathbb{R} \ , \  x\in X
$$


## The Rewritten Master problem

$$
\max_{x,t} \, c^{\intercal} x + t
$$

$$
s.t. \quad t +\left(u^{k}\right)^{\intercal}Ax \geq b^{\intercal}u^{k} \quad \forall k \in K
$$

$$
\qquad \ \left(r^{j}\right)^{\intercal}Ax \geq b^{\intercal}r^{j} \quad \forall j \in J
$$

$$
t \in \mathbb{R} \ , \  x\in X
$$

## Initial relaxed Master Problem
The first four constraints imply $y_{j} \leq 1$ for $j = 1, . . . , 4$, for any choice of $x$.
Thus, we can get a very simple bound for the problem: set $y_{j} = 1$ for $j = 1, . . . , 4$. This gives a value of $28$ from the $y$ variables.



In [127]:
A = np.array([[0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              [-1, -1],
              [-1, 0],
              [-1, 0],
              [0, -1],
              [0, -1],])

G = np.array([[1,0,1,0], 
              [1,0,0,1],
              [0,1,1,0],
              [0,1,0,1],
              [0,0,0,0],
              [1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
             ])
b = np.array([1,1,1,1,-1,0,0,0,0])

c = np.array([-15.0,-10.0])

h = np.array([8,9,5,6])

t = np.inf

In [128]:
#obj

QUBO_matrix_initial = np.zeros((2,2))
Var_dic, length = make_dictionary()

# OBJ
QUBO = np.diag(np.abs(c))  

# Select a solver
sampler = LeapHybridSampler()

sampleset = sampler.sample_qubo(QUBO)


In [167]:
OP = sampleset.to_pandas_dataframe()
OP_final = OP.to_numpy()[0]
result = OP_final[:length]
print(OP)
print(OP_final)
print("QUBO Q = :")
print( QUBO )
print("result x = :",result)

   0  1  2  3  4  5  6   energy  num_occurrences
0  1  1  1  1  0  1  1 -19055.0                1
[ 1.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00  1.0000e+00
  1.0000e+00 -1.9055e+04  1.0000e+00]
QUBO Q = :
[[-16965.   5760.   -170.   -340.   -170.   -340.   -680.]
 [  5610. -13010.   -110.   -220.   -110.   -220.   -440.]
 [  -170.   -110.    510.     20.      0.      0.      0.]
 [  -340.   -220.     20.   1040.      0.      0.      0.]
 [  -170.   -110.      0.      0.    450.     20.     40.]
 [  -340.   -220.      0.      0.     20.    920.     80.]
 [  -680.   -440.      0.      0.     40.     80.   1920.]]
result x = : [1. 1.]


In [130]:
x = result
obj = b-np.dot(A,x)

print("obj:",obj)

obj: [ 1.  1.  1.  1. -1.  0.  0.  0.  0.]


In [131]:
from scipy.optimize import linprog

In [132]:
#obj = [+1, +1, 1, 1, -1, 0, 0, 0, 0]
#      ─┬  ─┬  
#       │   │  .......
#       │   └┤ Coefficient for u2
#       └────┤ Coefficient for u1

lhs_ineq = [[-1, -1, -0, -0, -0, -1, -0, -0, -0],
            [-0, -0, -1, -1, -0, -0, -1, -0, -0],
            [-1, -0, -1, -0, -0, -0, -0, -1, -0],
            [-0, -1, -0, -1, -0, -0, -0, -0, -1]]

rhs_ineq = [-8,
            -9,
            -5,
            -6]

bnd = [(0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),   
       (0, float("inf")),]  # Bounds of u s

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,method="revised simplex")

In [133]:
print(opt)

     con: array([], dtype=float64)
     fun: -inf
 message: 'If feasible, the problem is (trivially) unbounded due  to a zero column in the constraint matrices. If you wish to check whether the problem is infeasible, turn presolve off.'
     nit: 0
   slack: array([nan, nan, nan, nan])
  status: 3
 success: False
       x: array([ 0.,  0.,  0.,  0., inf,  0.,  0.,  0.,  0.])


In [134]:
r = np.sign(opt['x'])
print(r)

[0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [135]:
cons_add_lhs = dot(r,A)
cons_add_rhs = dot(b,r)
print(cons_add_lhs,"<=",cons_add_rhs)

[-1. -1.] <= -1.0


In [136]:
symbol = "<="
if np.sign(cons_add_rhs) == -1:
    symbol = ">="

cons_add_lhs_final = np.sign(cons_add_rhs) * cons_add_lhs
cons_add_rhs_final = np.sign(cons_add_rhs) * cons_add_rhs


print(cons_add_lhs_final, symbol, cons_add_rhs_final)

[1. 1.] >= 1.0


In [137]:
#np.asarray(0.5 * (np.outer(cons_add_lhs_final,cons_add_lhs_final) - 3.0 * np.diag(cons_add_lhs_final)))

In [138]:
1.0 == 1

True

In [139]:
#function set
%run Functions_Set.ipynb

# Obj
QUBO_pre = np.diag(np.abs(c))  

# add constraint (r1)^{T} Ax ≤ b^{T} r1 to (RMP):

Penalty = np.max(np.abs(c))*10

#QUBO += Penalty * np.asarray(0.5 * (np.outer(cons_add_lhs_final,cons_add_lhs_final) - 3.0 * np.diag(cons_add_lhs_final)))

C = c
t = 0
new_Constraint = cons_add_lhs_final
new_Symbol = symbol
New_rhs = cons_add_rhs_final
round_num = 2 

print(C, t, new_Constraint, new_Symbol, New_rhs, round_num)

QUBO, _ = solve_master_old(C, t, QUBO_pre, new_Constraint, new_Symbol, New_rhs, round_num, Penalty, _print = 1)



print(QUBO)

#show_graph(QUBO)




[-15. -10.] 0 [1. 1.] >= 1.0 2
1.0 1
[1. 1.] >= 1.0
yep enter
[1. 1.] >= 1.0
label =>  0 , penalty is 150.0
>=1 
[[-135.  150.]
 [   0. -140.]]


In [140]:

# Select a solver
sampler = LeapHybridSampler()

sampleset = sampler.sample_qubo(QUBO)

In [141]:
OP = sampleset.to_pandas_dataframe()
OP_final = OP.to_numpy()[0]
result = OP_final[:length]
print("result x = :",result)

result x = : [0. 1.]


In [142]:
x = result
obj = b-np.dot(A,x)
print(obj)

[1. 1. 1. 1. 0. 0. 0. 1. 1.]


In [143]:
x = result
obj = b-np.dot(A,x)
print(obj)

[1. 1. 1. 1. 0. 0. 0. 1. 1.]


In [144]:
print(lhs_ineq)
print(rhs_ineq)

[[-1, -1, 0, 0, 0, -1, 0, 0, 0], [0, 0, -1, -1, 0, 0, -1, 0, 0], [-1, 0, -1, 0, 0, 0, 0, -1, 0], [0, -1, 0, -1, 0, 0, 0, 0, -1]]
[-8, -9, -5, -6]


In [145]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,method="revised simplex")

In [146]:
print(opt)

     con: array([], dtype=float64)
     fun: 11.0
 message: 'Optimization terminated successfully.'
     nit: 0
   slack: array([0., 0., 0., 0.])
  status: 0
 success: True
       x: array([0., 0., 0., 0., 0., 8., 9., 5., 6.])


In [161]:
t = 22
r = opt['x']
cons_add_lhs = dot(r,A)
cons_add_rhs = dot(b,r) - t
print(cons_add_lhs,"<=",cons_add_rhs)

[-17. -11.] <= -22.0


In [162]:
symbol = "<="
if np.sign(cons_add_rhs) == -1:
    symbol = ">="
turbulance = 1e-10
    
cons_add_lhs_final = np.sign(cons_add_rhs + turbulance) * cons_add_lhs
cons_add_rhs_final = np.sign(cons_add_rhs + turbulance) * cons_add_rhs


print(cons_add_lhs_final, symbol, cons_add_rhs_final)

[17. 11.] >= 22.0


In [163]:
#function set
%run Functions_Set.ipynb

# Obj
#QUBO = np.diag(np.abs(c))  

# add constraint (r1)^{T} Ax ≤ b^{T} r1 to (RMP):

new_Constraint = cons_add_lhs_final
new_Symbol = symbol
New_rhs = cons_add_rhs_final
round_num = 3

Penalty = np.max(np.abs(obj))*10

QUBO_pre = QUBO

QUBO, _ = solve_master_old(C, t, QUBO_pre, new_Constraint, new_Symbol, New_rhs, round_num, Penalty, _print = 1)



#print(QUBO)

#show_graph(QUBO)



22.0 1
[17. 11.] >= 22.0
yep enter
[17. 11.] >= 22.0
QUBO_Cons_ineq_normal
label >=  0 , penalty is 10.0
symbol == >=
Slack_Var_Add_num: 3.0
Slack_Var_Add_coeff: [1. 2. 4.]
var_matched_2: [0. 0. 0. 0. 1. 2. 4.]
normal else


In [164]:
# Select a solver
sampler = LeapHybridSampler()

sampleset = sampler.sample_qubo(QUBO)

OP = sampleset.to_pandas_dataframe()
OP_final = OP.to_numpy()[0]
result = OP_final[:length]
print("result x = :",result)

result x = : [1. 1.]


In [165]:
master_value = t + dot(c,result)
print(master_value,c,result)

-3.0 [-15. -10.] [1. 1.]


In [124]:
x = result
obj = b-np.dot(A,x)
print(obj)

[1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [120]:
print(lhs_ineq)
print(rhs_ineq)

[[-1, -1, 0, 0, 0, -1, 0, 0, 0], [0, 0, -1, -1, 0, 0, -1, 0, 0], [-1, 0, -1, 0, 0, 0, 0, -1, 0], [0, -1, 0, -1, 0, 0, 0, 0, -1]]
[-8, -9, -5, -6]


In [121]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,method="revised simplex")
print(opt)

     con: array([], dtype=float64)
     fun: 17.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([0., 0., 0., 0.])
  status: 0
 success: True
       x: array([2., 6., 3., 0., 0., 0., 6., 0., 0.])


In [93]:
r = opt['x']
cons_add_lhs = dot(r,A)
cons_add_rhs = dot(b,r)
print(cons_add_lhs,"<=",cons_add_rhs)

[-6.  0.] <= 11.0


In [63]:
#r = np.array([8,0,0,9,0,0,0,0,0])
cons_add_lhs = dot(r,A)
cons_add_rhs = dot(b,r)
print(cons_add_lhs,"<=",cons_add_rhs)

[-6.  0.] <= 11.0


In [ ]:
dot(b - dot(A, r)

In [64]:
symbol = "<="
if np.sign(cons_add_rhs) == -1:
    symbol = ">="
turbulance = 1e-10
    
cons_add_lhs_final = np.sign(cons_add_rhs + turbulance) * cons_add_lhs
cons_add_rhs_final = np.sign(cons_add_rhs + turbulance) * cons_add_rhs


print(cons_add_lhs_final, symbol, cons_add_rhs_final)

[-6.  0.] <= 11.0


In [65]:
#function set
%run Functions_Set.ipynb

# Obj
#QUBO = np.diag(np.abs(c))  

# add constraint (r1)^{T} Ax ≤ b^{T} r1 to (RMP):

new_Constraint = cons_add_lhs_final
new_Symbol = symbol
New_rhs = cons_add_rhs_final
round_num = 3


Penalty = np.max(np.abs(obj))*10

QUBO_pre = QUBO

QUBO, _ = solve_master_old(C, t, QUBO_pre, new_Constraint, new_Symbol, New_rhs, round_num, Penalty, _print = 1)



print(QUBO)

#show_graph(QUBO)

# Select a solver
sampler = LeapHybridSampler()

sampleset = sampler.sample_qubo(QUBO)

OP = sampleset.to_pandas_dataframe()
OP_final = OP.to_numpy()[0]
result = OP_final[:length]
print("result x = :",result)

11.0 0
[-6.  0.] <= 11.0
yep enter
[-6.  0.] <= 11.0
QUBO_Cons_ineq_normal
label >=  0 , penalty is 10.0
symbol == <=
Slack_Var_Add_num: 5.0
Slack_Var_Add_coeff: [ 1.  2.  4.  8. 16.]
var_matched_1: [-6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
var_matched_2: [ 0.  0.  0.  0.  0.  0.  0.  1.  2.  4.  8. 16.]
normal else
[[-5085.  2020.     0.     0.     0.     0.     0.   -60.  -120.  -240.
   -480.  -960.]
 [ 1870. -5090.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.]
 [    0.     0.  -330.    20.    40.    80.   160.     0.     0.     0.
      0.     0.]
 [    0.     0.    20.  -640.    80.   160.   320.     0.     0.     0.
      0.     0.]
 [    0.     0.    40.    80. -1200.   320.   640.     0.     0.     0.
      0.     0.]
 [    0.     0.    80.   160.   320. -2080.  1280.     0.     0.     0.
      0.     0.]
 [    0.     0.   160.   320.   640.  1280. -2880.     0.     0.     0.
      0.     0.]
 [  -60.     0.     0.     0.     0.     0.     0. 